# MSMH5 file

## Using the native MSMH5 OpenMM reporter

In [1]:
import openmm as mm
from openmm import app
from openmm import unit

import molsysmt as msm
from molsysmt import pyunitwizard as puw

In [2]:
molsys = msm.convert('4POC')
molsys = msm.extract(molsys, selection='molecule_type=="protein"')
molsys = msm.build.add_missing_hydrogens(molsys)
molsys = msm.build.solvate(molsys)

topology = msm.convert(molsys, 'openmm.Topology')
coordinates = msm.get(molsys, coordinates=True)
positions = puw.convert(coordinates[0], to_form='openmm.unit')

In [3]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')
system = forcefield.createSystem(topology, nonbondedMethod=app.PME,
        nonbondedCutoff=1.2*unit.nanometer, constraints=app.HBonds)
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)
platform = mm.Platform.getPlatformByName("CUDA")
simulation = app.Simulation(topology, system, integrator, platform)
simulation.context.setPositions(positions)
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

In [4]:
from molsysmt.thirds.openmm.reporters import MSMH5Reporter, TQDMReporter

In [5]:
tqdm_reporter = TQDMReporter(1000, 50000)
simulation.reporters.append(tqdm_reporter)

In [6]:
msmh5_reporter = MSMH5Reporter('traj.msmh5', 1000, steps=50000, selection='all',
            topology=topology, time=True, box=True, coordinates=True, velocities=False,
            potentialEnergy=True, kineticEnergy=True, temperature=True,
            includeInitialContext=True, constantReportInterval=True,
            constantStepSize=True, constantBox=True,
            auto_close=True)
simulation.reporters.append(msmh5_reporter)

In [7]:
simulation.step(50000)

  0%|          | 0/50000 [00:00<?, ?it/s]


Potential energy: -534329.36 kJ/mol ± 20050.53 kJ/mol
Temperature: 299.14 K ± 4.11 K

Execution time: 0 days, 0 hours, 0 minutes, and 55.76 seconds (154.961 ns/day).



## A first glance at the msmh5 file

In [9]:
aa = msm.convert('traj.msmh5', 'molsysmt.MSMH5FileHandler')

In [17]:
'topology' in aa.file

True

In [16]:
aa.file['topology']['atoms'].attrs['n_atoms']

44527

In [8]:
msm.get('traj.msmh5', n_lipids=True)

NotImplementedError: 

## Extract a new msmh5 file